# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv('/datasets/users_behavior.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Данные открыты.
Видим, что пользователей перешедших на тариф Ультра сильно меньше, отсюда вывод, что выборка не сбалансирована.
Целевым признаком для нашей задачи будет is_ultra. Постараемся обучить нашу модель для подбора необходимого тарифа пользователю.

## 2. Разбейте данные на выборки

Разобъем данные на train и valid,test. Размеры валидационного и тестового набора данных сделаем равными. 

In [4]:
df_train, df_valid=train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test=train_test_split(df_valid, test_size=0.5,random_state=12345)

Получили соотношение 60%, 20%,20%.


## 3. Исследуйте модели

Для исследования применим изученные ранее модели :
- дерево решений
- случайный лес
- логистическую регрессию

In [5]:
features_train=df_train.drop(['is_ultra'], axis=1)
target_train=df_train['is_ultra']

features_valid=df_valid.drop(['is_ultra'], axis=1)
target_valid=df_valid['is_ultra']

In [6]:
def accuracy_score_valid(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

**Исследование дерева решений**

In [7]:
for max_depth in range(1,21,2):
    model=DecisionTreeClassifier(max_depth=max_depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid=model.predict(features_valid)
    accuracy=accuracy_score(target_valid, predictions_valid)
    print(f"max_depth = {max_depth}:\t{accuracy_score_valid(model)}")

max_depth = 1:	0.7542768273716952
max_depth = 3:	0.7853810264385692
max_depth = 5:	0.7791601866251944
max_depth = 7:	0.7822706065318819
max_depth = 9:	0.7822706065318819
max_depth = 11:	0.7620528771384136
max_depth = 13:	0.7558320373250389
max_depth = 15:	0.7465007776049767
max_depth = 17:	0.7356143079315708
max_depth = 19:	0.7278382581648523


Исходя из результатов можно сделать вывод,что высокая оценки правильности равна 0.7822706065318819, когда задан гиперпараметр равный 7 или 9.

**Случайный лес**

In [8]:
for estimator in range(10,101,10):
    model=RandomForestClassifier(n_estimators=estimator, max_depth=7, random_state=12345)
    model.fit(features_train,target_train)
    print(f"n_estimators={estimator}:\t{accuracy_score_valid(model)}")
    #print(estimator,accuracy_score_valid(model))

n_estimators=10:	0.7947122861586314
n_estimators=20:	0.8009331259720062
n_estimators=30:	0.80248833592535
n_estimators=40:	0.80248833592535
n_estimators=50:	0.80248833592535
n_estimators=60:	0.7993779160186625
n_estimators=70:	0.8009331259720062
n_estimators=80:	0.7993779160186625
n_estimators=90:	0.7993779160186625
n_estimators=100:	0.8009331259720062


Модель случайного леса показывает лучший результат при 30 деревьях равный 0.80248833592535

**Логистическая регрессия**

In [9]:
def logistic_regression(solver='liblinear'):
    model=LogisticRegression(solver=solver,random_state=12345)
    model.fit(features_train,target_train)
    return accuracy_score_valid(model)


In [10]:
logistic_regression()

0.7589424572317263

Исходя из исследований моделей делаем вывод, что модель случайного леса дает наилучший результат.
Поэтому применим данную модель для нашей тестовой выборки.

## 4. Проверьте модель на тестовой выборке


In [11]:
features_test=df_test.drop(['is_ultra'],axis=1)
target_test=df_test['is_ultra']

model= RandomForestClassifier(n_estimators=70, max_depth=7, random_state=12345)
model.fit(features_train,target_train)

predictions_test=model.predict(features_test)
accuracy_score(target_test,predictions_test)

0.80248833592535

На тестовой выборке видим значения похожие на значения тренировочной выборки. Отсюда можно сделать вывод, что модель со своей задачей справляется и подбирает необходимый тариф для пользователей.

## 5. (бонус) Проверьте модели на адекватность

Если соотношение пропорций пользователей на тарифах для нашей модели и исходной выборки примерно похожи, то можно будет полагать, что модель со своей задачей более чем справляется и ее можно считать адекватной.

Проверим.

In [12]:
(df['is_ultra']==0).sum()/df.shape[0]


0.693528313627878

In [13]:
(df_test['is_ultra']==0).sum() / df_test.shape[0]

0.6842923794712286

   Доля большего класса выборки равна ~0.68, что похоже на значение исходной выборки.
   Таким образом, можем считать модель адекватной для использования.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
